# Live Demo: Saptial Information Generator

You can try this feature with your own data in this page. 

:::{tip}
You can choose a particular dataset by its index and explore the geographic information recommendations provided. To proceed, follow these steps:
1. Click on the "rocket icon" located in the top-right corner.
2. Select the option labeled `Live Code` from the menu.
3. Once the environment is launched, you'll be able to manually execute **each** code cell.

For any hidden code cells, simply click on `Show code cell source` and subsequently click `run` within each respective cell section.
:::

Here're the information you may input:
* `title`: the title of your dataset
* `description`: description of your dataset
* `resource_names`: file names in your dataset
* `resource_descriptions`: the description of files in your dataset
* `organization_title`: the tilte of the affiliatedorganization 
* `organization_description`: the description of the affiliatedorganization 

```{warning} Notice
At least one of the fields should be completed. Leaving all of them empty is not permissible.
```

## Function Definations
Just expend it and click `run`

In [22]:
# Packages Import ============================================================
import requests

# NLP task model
from ckip_transformers.nlp import CkipNerChunker
ner_driver = CkipNerChunker(model="bert-base")

# Function Definstion =========================================================
def search_osm_place(query):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": query,
        "format": "json",
        "polygon_geojson": "1",  # Request GeoJSON polygons
        "limit": 7
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return None

def make_keyword_map(input_list):
    # NER task
    ner = ner_driver(input_list)

    # Build keyword_map to store potential words
    avoid_class = ['QUANTITY', 'CARDINAL', 'DATE', 'ORDINAL']
    keyword_map = {}
    for sentence_ner in ner:
        for entity in sentence_ner:
            if(entity[1] in avoid_class):
                continue
        keyword_map[entity[0]] = entity[1]
    
    return keyword_map

def gen_keyword(title, description, resource_names, resource_descriptions, organization_title, organization_description):
    input_list = [title, description, resource_names, resource_descriptions, organization_title, organization_description]

    if all(not item for item in input_list):
        return -1
    else:
       keyword_map = make_keyword_map(input_list)
       return keyword_map

def output(result):
    if(result == -1):
        print("At least one of the fields should be completed. Leaving all of them empty is not permissible.")
        return
    else:
        for item in result:
            respond = search_osm_place(item)
            if respond:
                print(f"OSM result for {item} is:")
                for place in respond:
                    print("📍", place["display_name"])
                    print(str(place["geojson"]).replace("'", "\""))
                print('-------------------------------------------')
            else:
                    print("No geoInfo provided.")

### Input
✨ You can type information of your own dataset here:

In [29]:
title = ''
description = ''
resource_names = []
resource_descriptions = []
organization_title = ""
organization_description = ""

result = gen_keyword(title, description, resource_names, resource_descriptions, organization_title, organization_description)

### Output
Below's our recomnned wikidata keyword(s) for your dataset:

In [24]:
output(result)

OSM result for 文山區 is:
📍 文山區, 臺北市, 臺灣
{"type": "Polygon", "coordinates": [[[121.530556, 24.9979126], [121.5307611, 24.9974906], [121.5310765, 24.996546], [121.5313715, 24.9954353], [121.5314278, 24.9953209], [121.5316543, 24.9947062], [121.5317584, 24.9940135], [121.5317609, 24.99328], [121.5313799, 24.9925732], [121.5315576, 24.9922089], [121.5316762, 24.9918284], [121.5317501, 24.9910332], [121.5317011, 24.9905022], [121.5322879, 24.9908692], [121.5324338, 24.9910291], [121.5328032, 24.9915416], [121.5334249, 24.9921507], [121.5336966, 24.992438], [121.5340405, 24.9928683], [121.5341045, 24.9929415], [121.5343831, 24.9934076], [121.5345288, 24.9938208], [121.534915, 24.9949387], [121.5349599, 24.9950329], [121.5350673, 24.9951025], [121.5351682, 24.9951405], [121.5352376, 24.9951224], [121.5354555, 24.9949777], [121.535599, 24.9948456], [121.5357041, 24.9947281], [121.5358847, 24.9945681], [121.5359916, 24.9943849], [121.5362299, 24.9941215], [121.5365558, 24.993536], [121.5368734, 2

### Preview
Here you can select one of the geographic information, input it below to preview this location:

In [27]:
geoInfo = ''

In [28]:
import folium

center_coords = [25.041415686746607, 121.61472689731077]  # Sinica
m = folium.Map(location=center_coords, zoom_start=12)

if(len(geoInfo) == 0):
    print("please paste the geoJSON in the 'geoInfo' string.")
else:
    geojson = eval(geoInfo)
    folium.GeoJson(geojson).add_to(m)
    m.fit_bounds(m.get_bounds())
    display(m)

please paste the geoJSON in the 'geoInfo' string.
